## Variational principle for extended shallow water

\begin{align}
\mathscr{L} &= \int_{t_0}^{t_1}\textrm{d}t
\int_{-h}^0\textrm{d}z
\int_{x_0}^{x_1}\textrm{d}x
\left(
\frac{\rho_w}2\big(u_t^2+w_t^2\big)
+p(u_x+w_z)
-\rho_wgw
\right)\\
&\quad+\int_{t_0}^{t_1}\textrm{d}t
\int_{x_0}^{x_1}\textrm{d}x
\left(
\frac{\rho_wg}2\big(\eta^2-2\eta w|_{z=0}\big)
+\big[\tau_{22}w\big]_{-h}^0
\right)\\
&\quad+\int_{t_0}^{t_1}\textrm{d}t
\int_{-h}^0\textrm{d}z
\big[\tau_{11}u\big]_{x_0}^{x_1}
\end{align}

\begin{align}
0 &= \delta\mathscr{L}\\
&= \int_{t_0}^{t_1}\textrm{d}t
\int_{-h}^0\textrm{d}z
\int_{x_0}^{x_1}\textrm{d}x
\left(
\rho_w\big(u_t\delta u_t+w_t\delta w_t\big)
+p(\delta u_x+\delta w_z)
+\delta p(u_x+w_z)
-\rho_wg\delta w
\right)\\
&\quad+\int_{t_0}^{t_1}\textrm{d}t
\int_{x_0}^{x_1}\textrm{d}x
\Big(
\rho_wg\delta\eta\big(\eta- w|_{z=0}\big)
+\big(\tau_{22}|_{z=0}-\rho_wg\eta\big)\delta w|_{z=0}\\
&\qquad\qquad\qquad\qquad
-\tau_{22}|_{z=-h}\delta w|_{z=-h}
-w|_{z=-h}\delta \tau_{22}|_{z=-h}
\Big)\\
&\quad+\int_{t_0}^{t_1}\textrm{d}t
\int_{-h}^0\textrm{d}z
\big[\tau_{11}\delta u + \delta\tau_{11}u\big]_{x_0}^{x_1}
\end{align}

\begin{align}
0 &= \int_{t_0}^{t_1}\textrm{d}t
\int_{-h}^0\textrm{d}z
\int_{x_0}^{x_1}\textrm{d}x
\left(
-\delta u\big(\rho_w u_{tt}+p_x\big)
-\delta w\big(\rho_w w_{tt}+p_z +\rho_wg\big)
+\delta p(u_x+w_z)
\right)\\
&\quad+\int_{t_0}^{t_1}\textrm{d}t
\int_{x_0}^{x_1}\textrm{d}x
\Big(
\rho_wg\delta\eta\big(\eta- w|_{z=0}\big)
+\big(\tau_{22}|_{z=0}+p|_{z=0}-\rho_wg\eta\big)\delta w|_{z=0}\\
&\qquad\qquad\qquad\qquad
-\big(\tau_{22}|_{z=-h}+p_{z=-h}\big)\delta w|_{z=-h}
-w|_{z=-h}\delta \tau_{22}|_{z=-h}
\Big)\\
&\quad+\int_{t_0}^{t_1}\textrm{d}t
\int_{-h}^0\textrm{d}z
\big[\big(\tau_{11}+p\big)\delta u + \delta\tau_{11}u\big]_{x_0}^{x_1}
\end{align}

The full governing equations are the usual momentum equations and the incompressibility terms
\begin{align}
0 &= \rho_w u_{tt}+p_x,\\
0 &= \rho_w w_{tt}+p_z +\rho_wg,\\
0 &= u_x+w_z,
\end{align}
and we also have horizontal boundary conditions
\begin{align}
0 &= \eta-w|_{z=0},\\
0 &= \tau_{22}|_{z=0}+p|_{z=0}-\rho_wg\eta\\
&= -p_a+p|_{z=0}-\rho_wg\eta,\\
0 &= w|_{z=-h},
\end{align}

We now make the approximations
\begin{align}
u&=U(x,t),\\
w&=(z+h)\frac{\eta}h,
\end{align}
and to satisfy as many of the governing equations as possible we also require
\begin{align}
\eta&=-hU_x,\\
p&=p|_{z=0}-\rho_wgz-\int_{0}^z\big(\rho_ww_{tt}\big)\textrm{d}z'\\
&=p_a + \rho_wg(\eta-z) - \rho_w(z^2+2zh)\frac{\eta_{tt}}{2h}\\
&=p_a - \rho_wg(hU_x + z) + \rho_w(z^2+2zh)\frac{U_{xtt}}{2}.
\end{align}

Then from the term corresponding to the horizontal momentum equation we get
\begin{align}
0&=\rho_whU_{tt}
-\rho_wgh^2U_{xx}
- \rho_w\frac{h^3}3U_{xxtt},
\end{align}
or as Porter (2019) writes it
\begin{align}
0& = gQ_{xx} + \frac{h}3Q_{xxtt} -\frac{1}{h}Q_{tt},
\end{align}
with $Q=hU$.

This is Diki's extended shallow water equation; if we neglect the vertical kinetic energy in the Lagrangian the final term disappears and the governing equation is the usual shallow water equation. The conditions at any vertical boundaries will be different though.

In [1]:
import numpy as np

from pywave.scattering.extended_shallow_water import ExtendedShallowWater
from pywave.scattering.medium_boundary import MediumBoundary
from pywave.utils.jacobi_polynomials import JacobiPolynomials
from collections import namedtuple

In [2]:
Jacobi = namedtuple('Jacobi', ['jp', 'z', 'w', 'h', 'ipm'])

In [3]:
class ESWJacobi(MediumBoundary):
    
    def __init__(self, lhs, rhs, max_degree=5, **kwargs):
        super().__init__(lhs, rhs, **kwargs)
        self.max_degree = max_degree
        hh = [med.depth for med in self.media]
        self.min_depth = np.min(hh)
        self.deeper_media = np.argmin(hh)
        jp = JacobiPolynomials(alpha=-1./3, beta=0, a=-self.min_depth, b=0)
        xwh = jp.quad_points_weights()
        ipm = jp.get_inner_product_matrix(xwh=xwh)
        self.jacobi = Jacobi(jp, *xwh, ipm)


#     def assemble(self):
        #lhs
        

In [4]:
esw1 = ExtendedShallowWater(depth=40, period=15)
esw2 = ExtendedShallowWater(depth=60, period=15)

In [5]:
bdy = ESWJacobi(esw1, esw2, max_degree=4)

In [6]:
vars(bdy)

{'position': 0,
 'media': [<pywave.scattering.extended_shallow_water.ExtendedShallowWater at 0x7f9ccc6c71d0>,
 'condition_types': [{'displacement': True}, {'displacement': True}],
 'max_degree': 4,
 'min_depth': 100,
 'deeper_media': 0,
 'jacobi': Jacobi(jp=<pywave.utils.jacobi_polynomials.JacobiPolynomials object at 0x7f9c730ce048>, z=array([-97.93410781, -89.43397892, -75.39702276, -57.79098638,
        -39.08807911, -21.91486771,  -8.68290357,  -1.24741544]), w=array([1.14221113, 2.58018351, 3.83526227, 4.80875109, 5.41289453,
        5.56336825, 5.14818058, 3.82566899]), h=array([32.31652035,  8.07913009,  4.61664576,  3.23165204,  2.48588618,
         2.01978252,  1.70086949,  1.46893274]), ipm=array([[ 0.0353445 ,  0.07984101,  0.11867807,  0.14880164,  0.16749621,
          0.17215245,  0.15930492,  0.11838122],
        [-0.13651012, -0.26312391, -0.28005669, -0.17648853,  0.01018289,
          0.20755999,  0.33259783,  0.30583854],
        [ 0.22066768,  0.27444641, -0.01348399

In [7]:
vars(bdy.jacobi.jp)

{'alpha': -0.3333333333333333, 'beta': 0, 'max_degree': 7, 'a': -100, 'b': 0}